In [19]:
import math
import pandas as pd
import geopandas as gpd
import folium
from folium import Marker
from folium.plugins import HeatMap, MarkerCluster
from geopy.geocoders import Nominatim  

In [4]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')


In [5]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv(r"C:\Users\mugok\Desktop\Data sets\starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [6]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [7]:
# Create the geocoder
geolocator = Nominatim(user_agent="Anzy_kenya")

# Your code here
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

In [22]:
# Create a base map
m_1 = folium.Map(location=[37.88,-122.26],tiles = 'openstreetmap' , zoom_start=13)

# Your code here: Add a marker for each Berkeley location
# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Uncomment to see a hint
#q_2.a.hint()

# Show the map
embed_map(m_1, 'Map_1.html')

In [9]:
CA_counties = gpd.read_file(r"C:\Users\mugok\Desktop\Data sets\CA_county_boundaries\CA_county_boundaries\CA_county_boundaries.shp")
CA_counties.crs = 'epsg:4326'
CA_counties.head()


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


In [10]:
#Create this three data frames
CA_pop = pd.read_csv(r"C:\Users\mugok\Desktop\Data sets\CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv(r"C:\Users\mugok\Desktop\Data sets\CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv(r"C:\Users\mugok\Desktop\Data sets\CA_county_median_age.csv", index_col="GEOID")

In [11]:
#Join the above to CA counties.
data = CA_pop.join([CA_high_earners,CA_median_age]).reset_index()
CA_stats = gpd.GeoDataFrame.merge(CA_counties,data)


In [13]:
#Create a new column - density.
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]
CA_stats.head()


,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


In [14]:
sel_counties = CA_stats.loc[((CA_stats['high_earners'] >=100000 )&
                            (CA_stats['median_age'] < 38.5) &
                             (CA_stats['density'] >=285) & 
                            ((CA_stats['high_earners'] >=500000 )|
                             (CA_stats['median_age'] <35.5 )|
                             (CA_stats['density'] >=1400 )))]


In [15]:
sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


In [16]:
#Create Starbucks_gdf GeoDataFrame
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = 'epsg:4326'
starbucks_gdf.head()

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.40000 34.51000)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000)


In [17]:
#Number of stores in the counties selected
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)
print(num_stores)

locations_of_interest.head()

1043


,Store Number,Store Name,Address,City,Longitude,Latitude,geometry,index_right,GEOID,name,area_sqkm,population,high_earners,median_age,density
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
14,76365-97162,Target Alhambra T-184,1220 West Main Street Alhambra CA,Alhambra,-118.14,34.09,POINT (-118.14000 34.09000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
15,6794-41839,Fremont Ave & Mission Rd,"1131 S Fremont Ave, A Alhambra CA",Alhambra,-118.15,34.08,POINT (-118.15000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
16,11220-104633,"Atlantic & Valley, Alhambra",1410 South Atlantic Blvd. Alhambra CA,Alhambra,-118.13,34.08,POINT (-118.13000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834


In [23]:
# Visualize the store locations.
# Create a base map
m_2 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
#for idx, row in locations_of_interest.iterrows():
    #Marker([row['Latitude'], row['Longitude']]).add_to(m_6)
    # Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_2.add_child(mc)


In [24]:
# Show the map
embed_map(m_2, 'Map_2.html')